In [ ]:
psycho_sound.stop()

# Psychopy training first script
- 11.11.2025
- the environment : `conda activate opt/psychopy`


In [ ]:
# Imports
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent.parent))  # Adjust the path as needed

from psychopy import prefs
from psychopy import hardware
from psychtoolbox import PsychPortAudio
from psychopy import sound, core
from subcorticalSTRF.stim_generator.soundgen import SoundGen
# SoundGen.calculate_num_tones 
# SoundGen.sound_maker 
SoundGen.ramp_in_out
SoundGen.sine_ramp
SoundGen.sample_frequencies_gaussian
SoundGen.generate_sequence_from_freq_array


# Instantiate the sound generator class
sound_generator = SoundGen(sample_rate=44100, tau=0.005)


In [ ]:
# Set the parameters for stimuli generation
tone_duration = 0.2 # Duration of each tone in seconds
freq = 440  # Fundamental frequency in Hz
num_harmonics = 1  # Number of harmonics
harmonic_factor = 0.5  # Amplitude scaling factor for harmonics
my_tone = sound_generator.sound_maker(freq, num_harmonics, tone_duration, harmonic_factor)
Fs = sound_generator.sample_rate

# Play the generated tone using PsychPortAudio

#pahandle = PsychPortAudio('Open', [, deviceid][, mode][, reglatencyclass][, freq][,channels][, buffersize][, suggested latency][, selectchannels][, specialFlags=0])  # Default latency mode
pahandle = PsychPortAudio('Open', [], [], [0], Fs, 2)
PsychPortAudio('FillBuffer', pahandle, my_tone)
PsychPortAudio('Start', pahandle)




``` python
pahandle = PsychPortAudio('Open' [, deviceid][, mode][, reqlatencyclass][, freq][, channels][, buffersize][, suggestedLatency][, selectchannels][, specialFlags=0])
``     


Open a PortAUdio audio device and initialize it. Returns a 'pahandle' device handle for the device.
On most operating systems you can open each physical sound device only once per running session. (If you need to call 'Open' multiple times on the same audio device, read the section about *slave* devices.)

deviceid = 0.0
mode: 1 (default, sound playback only)
reqlatencyclass: level 3
freq: requested playback/capture rate in samples per second. defaults a value that depends on the requested latency modoe. 
'channels': default 2, for stereo



``` python
[underflow, nextSampleStartIndex, nextSampleETASecs] = PsychPortAudio('FillBuffer', pahandle, bufferdata [, streamingrefill=0][, startIndex=Append]); 
```
`bufferdata` is usualy a NumPy 2D matrix with audito data in (ideally) float32 format, or also float64 format. Each column of the matrix specifies one sound channel, each row one sample for each channel. Only floating point values are supported. Samples need to be in range -1.0 to +1.0, with 0.0 for silence. 


PsychPortAudio('Close' [, pahandle]);

bufferhandle = PsychPortAudio('CreateBuffer' [, pahandle], bufferdata);

[underflow, nextSampleStartIndex, nextSampleETASecs] = PsychPortAudio('FillBuffer', pahandle, bufferdata [, streamingrefill=0][, startIndex=Append]);

PsychPortAudio('DeleteBuffer'[, bufferhandle] [, waitmode]);

startTime = PsychPortAudio('Start', pahandle [, repetitions=1] [, when=0] [, waitForStart=0] [, stopTime=inf] [, resume=0]);
Start a PortAudio audio device. The 'pahandle' is the handle of the device to
start. Starting a device means: Start playback of output devices, start
recording on capture device, do both on full duplex devices. 'waitForStart' if
set to 1 will wait until device has really started, default is to continue
immediately, ie. only schedule start of device. 'when' Requested time, when
device should start. Defaults to zero, i.e. start immediately. If set to a
non-zero system time, PTB will do its best to start the device at the requested
time, but the accuracy of start depends on the operating system, audio hardware
and system load. If 'waitForStart' is set to non-zero value, ie if PTB should
wait for sound onset, then the optional return argument 'startTime' will contain
an estimate of when the first audio sample hit the speakers, i.e., the real
start time.

The 'repetitions' parameter defines how often the playback of the sound data
should be repeated. A setting of zero will cause infinite repetitions, ie.,
until manually stopped via the 'Stop' subfunction. A positive setting will cause
the provided number of repetitions to happen. The default setting is 1, ie.,
play exactly once, then stop. Fractional values are allowed, e.g, 1.5 for one
and a half repetition.


status = PsychPortAudio('GetStatus' pahandle);
status = PsychPortAudio('GetStatus', pahandle);

Returns 'status', a struct with status information about the current state of
device 'pahandle'.
The struct contains the following fields:
Active: Can be 1 if playback or recording is active, or 0 if playback/recording
is stopped or not yet started.
RequestedStartTime: Is the requested start time of the audio stream after start
of playback/recording. StartTime: Is the real start time of the audio stream
after start of playback/recording. If both, playback and recording are active
(full-duplex mode), it is the start time of sound playback, ie an estimate of
when the first sample hit the speakers. Same goes for pure playback. 
CaptureStartTime: Start time of audio capture (if any is active) - an estimate
of when the first sound sample was captured. In pure capture mode, this is
nearly identical to StartTime, but whenever playback is active, StartTime and
CaptureStartTime will differ. CaptureStartTime doesn't take the user provided
'LatencyBias' into account.
RequestedStopTime: The requested stop / sound offset time for playback of
sounds, as selected via the 'stopTime' paramter in the 'Start',
'RescheduleStart' or 'Stop' function. This will show a very large (~ infinite)
value if no stop time has been sprecified.
EstimatedStopTime: Estimated time when sound playback has stopped. This is an
estimate of when exactly the last audio sample will leave the speaker. The value
is zero as long as the estimate isn't available. Due to the latency involved in
sound playback, the value may become available a few msecs before or after
actual sound offset.
CurrentStreamTime: Estimate of when the most recently submitted sample will hit
the speaker. This corresponds roughly to 'PositionSecs' below, but in absolute
realtime.
ElapsedOutSamples: Total number of samples played out since start of playback.
This count increments monotonically from start of playback to stop of playback.
This denotes the absolute sample position that will hit the speaker at time
'CurrentStreamTime'. 
PositionSecs is an estimate of the current stream playback position in seconds
within the current playback loop of the current buffer. it's not totally
accurate, because it measures how much sound has been submitted to the sound
system, not how much sound has left the speakers, i.e., it doesn't take driver
and hardware latency into account.
SchedulePosition: Current position in a running schedule, if any.
XRuns: Number of dropouts due to buffer overrun or underrun conditions. This is
not perfectly reliable, as the algorithm can miss some dropouts. Iow.: A
non-zero or increasing value means that audio glitches during playback or
capture happened, but a zero or constant value doesn't mean everything was
glitch-free, because some glitches can't get reliably detected on some operating
systems or audio hardware.
TotalCalls, TimeFailed and BufferSize are only for debugging of PsychPortAudio
itself.
CPULoad: How much load does the playback engine impose on the CPU? Values can
range from 0.0 = 0% to 1.0 for 100%. Values close to 1.0 indicate that your
system can't handle the load and timing glitches or sound glitches are likely.
In such a case, try to reduce the load on your system.
PredictedLatency: Is the latency in seconds of your driver+hardware combo. It
tells you, how far ahead of time a sound device must be started ahead of the
requested onset time via PsychPortAudio('Start'...) to make sure it actually
starts playing in time. High quality systems like Linux or MacOS/X may allow
values as low as 5 msecs or less on standard hardware. Other operating systems
may require dozens or hundreds of milliseconds of headstart. Caution: In
full-duplex mode, this value only refers to the latency on the sound output, not
in the sound input! Also, this is just an estimate, not 100% reliable.
LatencyBias: Is an additional bias setting you can impose via
PsychPortAudio('LatencyBias', pahandle, bias); in case our drivers estimate is a
bit off. Allows fine-tuning.
SampleRate: Is the sampling rate for playback/recording in samples per second
(Hz).
OutDeviceIndex: Is the deviceindex of the playback device, or -1 if not opened
for playback. You can pass OutDeviceIndex to PsychPortAudio('GetDevices', [],
OutDeviceIndex); to query information about the device.
InDeviceIndex: Is the deviceindex of the capture device, or -1 if not opened for
capture.
RecordedSecs: Is the total amount of recorded sound data (in seconds) since
start of capture.
ReadSecs: Is the total amount of sound data (in seconds) that has been fetched
from the internal buffer. The difference between RecordedSecs and ReadSecs is
the amount of recorded sound data pending for retrieval. 



[startTime endPositionSecs xruns estStopTime] = PsychPortAudio('Stop', pahandle [,waitForEndOfPlayback=0] [, blockUntilStopped=1] [, repetitions] [, stopTime]);
[startTime endPositionSecs xruns estStopTime] = PsychPortAudio('Stop', pahandle [, waitForEndOfPlayback=0] [, blockUntilStopped=1] [, repetitions] [, stopTime]);

Stop a PortAudio audio device. The 'pahandle' is the handle of the device to
stop.
'waitForEndOfPlayback' - If set to 1, this method will wait until playback of
the audio stream finishes by itself. This only makes sense if you perform
playback with a defined playback duration. The flag will be ignored when
infinite repetition is requested (as playback would never stop by itself,
resulting in a hang) and if no scheduled 'stopTime' has been set, either in this
call or in the 'Start' function. The setting will be also ignored if this is a
pure recording session.
A setting of 0 (which is the default) requests stop of playback without waiting
for all the sound to be finished playing. Sound may continue to be played back
for multiple milliseconds after this call, as this is a polite request to the
hardware to finish up.
A setting of 2 requests abortion of playback and/or capture as soon as possible
with your sound hardware, even if this creates audible artifacts etc. Abortion
may or may not be faster than a normal stop, this depends on your specific
hardware, but our driver tries as hard as possible to get the hardware to shut
up. In a worst-case setting, the hardware would continue to playback the sound
data that is stored in its internal buffers, so the latency for stopping sound
would be the the same as the latency for starting sound as quickly as possible.
E.g., a 2nd generation Intel MacBook Pro seems to have a stop-delay of roughly
5-6 msecs under optimal conditions (e.g., buffersize = 64, frequency=96000, OS/X
10.4.10, no other sound apps running).
A setting of 3 will not try to stop the playback now: You can use this setting
if you just want to use this function to wait until playback stops by itself,
e.g. because the set number of 'repetitions' or the set 'stopTime' has been
reached. You can also use this to just change the 'repetitions' or 'stopTime'
settings without waiting for anything.


#List of tuples: (freq, num_harmonics, tone_duration, harmonic_factor, total_duration, isi)
# sequence_params = [
     (100, 4, 0.133, 0.5, 5, 0.033), # Condition 1
     (100, 4, 0.133, 0.5, 5, 0.200), # Condition 2
    (100, 4, 0.133, 0.5, 5, 0.867), # Condition 3
    #    (100, 4, 0.033, 0.5, 5, 0.133), # Condition 4 -- this almost always clicks
     (100, 4, 0.200, 0.5, 5, 0.133), # Condition 5
     (100, 4, 0.867, 0.5, 5, 0.133), # Condition 6
     (100, 4, 0.267, 0.5, 5, 0.067), # Condition 7
     (100, 4, 0.800, 0.5, 5, 0.200), # Condition 8
     (100, 4, 5.000, 0.1, 5, 0.000), # Condition 9
 ]


In [ ]:

# Create a sound object with specified parameters


# Play the sound
tone.play()
# Wait for the sound to finish playing
core.wait(tone.getDuration())



In [ ]:
tone.play()
# you can give a numpy array of Nx2 array of floats (-1.0 to 1.0), to specify the sound as a waveform

Sound.setSound(mysound, )

https://www.psychopy.org/_modules/psychopy/sound/backend_ptb.html#SoundPTB.setSound

The arguments of the sound function:
setSound(value, secs=0.5, octave=4, hamming=None, log=True)[source]
Set the sound to be played.

Often this is not needed by the user - it is called implicitly during initialisation.

Parameters:
value: can be a number, string or an array:
If it’s a number between 37 and 32767 then a tone will be generated at that frequency in Hz.

It could be a string for a note (‘A’, ‘Bfl’, ‘B’, ‘C’, ‘Csh’. …). Then you may want to specify which octave.

Or a string could represent a filename in the current location, or mediaLocation, or a full path combo

Or by giving an Nx2 numpy array of floats (-1:1) you can specify the sound yourself as a waveform

secs: duration (only relevant if the value is a note name or
a frequency value)

octave: is only relevant if the value is a note name.
Middle octave of a piano is 4. Most computers won’t output sounds in the bottom octave (1) and the top octave (8) is generally painful

# Actually psychtoolbox is included in the psychopy sound module, so you can use it directly

GetSecs
IOPort
PsychHID
PsychPortAudio
WaitSecs
psychtoolbox

I think these are the methods/functions that are available from psychtoolbox

Jasmin told she uses PsychPortAudio

```
from /home/ekim/opt/psychopy/lib/python3.10/site-packages/psychtoolbox/audio.py

RIGHT NOW THIS IS A WORK IN PROGRESS. ALTHOUGH THE PsychPortAudio LIB HAS BEEN
TESTED BY MARIO, this additional wrapper is under development. Use with extreme
caution only for development purposes right now.


e.g.::

    from psychtoolbox import PsychPortAudio
    pahandle = PsychPortAudio('Open', [], [], [0], Fs, 2)
    PsychPortAudio('FillBuffer', pahandle, stereowav)
    PsychPortAudio('Start', pahandle)

becomes::

    from psychtoolbox import audio
    stream = audio.Stream([], [], [0], Fs, 2)
    stream.fill_buffer(stereowav)
    stream.start()

(c) 2018 Jon Peirce - Licensed under MIT license.
"""

"""
Function summary from PsychPortAudio for not-yet-implemented functions:

pahandle = PsychPortAudio('OpenSlave', pamaster [, mode][, channels]
    [, selectchannels]);

PsychPortAudio('DeleteBuffer'[, bufferhandle] [, waitmode]);
PsychPortAudio('RefillBuffer', pahandle [, bufferhandle=0], bufferdata
    [, startIndex=0]);
PsychPortAudio('SetLoop', pahandle[, startSample=0][, endSample=max]
    [, UnitIsSeconds=0]);

PsychPortAudio('UseSchedule', pahandle, enableSchedule [, maxSize = 128]);
[success, freeslots] = PsychPortAudio('AddToSchedule', pahandle
    [, bufferHandle=0][, repetitions=1][, startSample=0][, endSample=max]
    [, UnitIsSeconds=0][, specialFlags=0]);
startTime = PsychPortAudio('RescheduleStart', pahandle, when [, waitForStart=0]
    [, repetitions] [, stopTime]);
"""
import atexit
from . import PsychPortAudio

def get_version_info():
    return PsychPortAudio('Version')


def verbosity(level=None):
    """Gets or sets the verbosity level (integer 1-5).

    If no level is provided the current level of verbosity is returned.
    If a new level is provided the old level setting is returned"""
    oldlevel = PsychPortAudio('Verbosity', level)


def get_open_device_count():
    return PsychPortAudio('GetOpenDeviceCount')


def get_devices(device_type=None, device_index=None):
    """Get a list of devices on the current system, optionally of certain types

    :param device_type:
    :param device_index:
    :return:
    """
    return PsychPortAudio('GetDevices', device_type, device_index)


def tune_engine(yield_interval, mutex_enable, lock_to_core1,
                audioserver_autosuspend):
    """Sets values for various advanced tuning parameters for the audio engine

    :param yield_interval:
    :param mutex_enable:
    :param lock_to_core1:
    :param audioserver_autosuspend:
    :return: the 4 values in their final state
    """
    return PsychPortAudio('EngineTunables', yield_interval, mutex_enable,
                          lock_to_core1, audioserver_autosuspend)


class Stream:
    """
    Creates a Psychtoolbox Stream with the given settings.
    See also http://psychtoolbox.org/docs/PsychPortAudio-Open
    """
    def __init__(self, device_id=[], mode=[], latency_class=[0],
                 freq=48000, channels=2,
                 buffer_size=[], suggested_latency=[], select_channels=[],
                 flags=0):
        # PsychPortAudio('Open', [], [], [0], Fs, 2);
        self.handle = PsychPortAudio('Open', device_id, mode,
                                     latency_class,
                                     freq, channels, buffer_size,
                                     suggested_latency, select_channels,
                                     flags)
        self._closed=False
        atexit.register(self.close)

    def start(self, repetitions=1, when=0, wait_for_start=0,
              stop_time=None, resume=0):
        """Start the stream for a given number of repetitions, potentially with
        a delayed start.

        :param repetitions:
        :param when:
        :param wait_for_start:
        :param stop_time:
        :param resume:
        :return:
        """
        start_time = PsychPortAudio('Start', self.handle, repetitions, when,
                                    wait_for_start, stop_time, resume)
        return start_time

    def stop(self, wait_for_end_playback=0, block_until_stopped=1,
             repetitions=None, stopTime=None):
        """
        :param wait_for_end_playback:
        :param block_until_stopped:
        :param repetitions:
        :param stopTime:
        :return: startTime, endPositionSecs, xruns, estStopTime
        """
        return PsychPortAudio('Stop', self.handle,
                              wait_for_end_playback, block_until_stopped,
                              repetitions, stopTime)

    def close(self):
        """Close the current stream"""
        if getattr(self, '_closed', False): # if we're already closed don't try again
            return
        try:
            PsychPortAudio('Close', self.handle)
            self._closed = True
        except Exception as err:
            if "Invalid audio device handle" in str(err):
                # this most likely means the stream is closed already
                return
            else:
                raise err
                ```

